In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import muon as mu

# Import a module with ATAC-seq-related functions
from muon import atac as ac

# Process ATAC-seq

In [ ]:
# change to you data file
mdata = mu.read("muon_data/pbmc10k.h5mu")
mdata

In [ ]:
atac = mdata.mod['atac']
atac

## Pre-processing

In [ ]:
sc.pp.calculate_qc_metrics(atac, percent_top=None, log1p=False, inplace=True)

In [ ]:
mu.pp.filter_var(atac, 'n_cells_by_counts', lambda x: x >= 10)
mu.pp.filter_obs(atac, 'n_genes_by_counts', lambda x: (x >= 2000) & (x <= 15000))
mu.pp.filter_obs(atac, 'total_counts', lambda x: (x >= 4000) & (x <= 40000))

In [ ]:
atac.obs['NS']=1

In [ ]:
ac.pl.fragment_histogram(atac, region='chr1:1-2000000')

In [ ]:
ac.tl.nucleosome_signal(atac, n=1e6)

In [ ]:
mu.pl.histogram(atac, "nucleosome_signal", kde=False)

In [ ]:
# TSS enrichment
ac.tl.get_gene_annotation_from_rna(mdata['rna']).head(3) 

In [ ]:
tss = ac.tl.tss_enrichment(mdata, n_tss=1000) 
tss

In [ ]:
ac.pl.tss_enrichment(tss)

In [ ]:
# normalization
atac.layers["counts"] = atac.X

In [ ]:
# may lead to kernel dead in some machine
sc.pp.normalize_per_cell(atac, counts_per_cell_after=1e4)

In [ ]:
sc.pp.log1p(atac)

In [ ]:
sc.pp.highly_variable_genes(atac, min_mean=0.05, max_mean=1.5, min_disp=.5)

In [ ]:
sc.pl.highly_variable_genes(atac)

In [ ]:
np.sum(atac.var.highly_variable)

In [ ]:
atac.raw = atac

## PCA / TSNE Mapping

### PCA

In [ ]:
sc.pp.scale(atac)
sc.tl.pca(atac)

In [ ]:
sc.pp.neighbors(atac, n_neighbors=10, n_pcs=30)

In [ ]:
sc.tl.leiden(atac, resolution=.5)

/var/folders/rn/34mqntmj6sn0b3gfqr91slqc0000gn/T/ipykernel_55189/937485634.py:1: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(atac, resolution=.5)


### TSNE

In [ ]:
sc.tl.tsne(atac)

In [ ]:
atac

## Save Data

In [ ]:
# change to your data file name
mu.write("muon_data/pbmc10k.h5mu/atac", atac)